In [1]:
import pickle 
import pandas as pd 
import numpy as np 
import os 
from glob import glob 
import tqdm 
from PIL import Image 
from tqdm import tqdm 
import wandb
import math

from sklearn.metrics import f1_score,accuracy_score

from src.Dataset import CifarDataset,label_unlabel_load,dataset_load
from src.Models import Model,PiModel
from src.Loss import PiCriterion

import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms

import warnings 
warnings.filterwarnings('ignore')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def valid(model,test_loader,cfg):
    labels = []
    y_preds = [] 
    model.eval() 
    for batch_imgs,batch_labels in test_loader:
        batch_imgs = batch_imgs.type(torch.float32).to(cfg['device'])
        with torch.no_grad():
            y_pred = model(batch_imgs,False)
        y_pred = torch.argmax(F.softmax(y_pred),dim=1)
        y_pred = y_pred.detach().cpu().numpy()
        
        y_preds.extend(y_pred)
        labels.extend(batch_labels.detach().cpu().numpy())    
    f1 = f1_score(np.array(y_preds),np.array(labels),average='macro')
    auc = accuracy_score(np.array(y_preds),np.array(labels))
    return f1, auc

In [3]:
cfg = {}
cfg['dataset'] = 'cifar10'
cfg['model_name'] = 'resnet18'
cfg['unlabel'] = 100
cfg['label'] = 1000
cfg['batch_size'] = 100 
cfg['device'] = 'cuda:0'
cfg['lr'] = 0.003 
cfg['beta1'] = 0.8
cfg['beta2'] = 0.999 
cfg['epochs'] = 300 
cfg['std'] = 0.15 
cfg['super_only'] = False
cfg['Early_stop'] = 50 
cfg['dir'] = 'test'

train_set,test_set = label_unlabel_load(cfg)

test_dataset  = CifarDataset(test_set,unlabel=False)

test_loader  = DataLoader(test_dataset,batch_size=cfg['batch_size'],shuffle=False)
    

In [4]:
dirs = sorted(glob('./Saved_models/*'))

best = []
last = [] 
for i in range(len(dirs)):
    best_model = torch.load(os.path.join(dirs[i],'best.pt'))
    try:
        last_model = torch.load(os.path.join(dirs[i],'last.pt'))
    except:
        pass
    f1_best , auc_best = valid(best_model,test_loader,cfg)
    f1_last , auc_last = valid(last_model,test_loader,cfg)
    
    best.append([f1_best,auc_best])
    last.append([f1_last,auc_last])

In [5]:
df = pd.Series(dirs).apply(lambda x : x.split('/')[-1].split('_'))
df= pd.DataFrame(np.array(list(df.values)).reshape(-1,4))
#df.columns = ['Method','Encoder','SuperOnly','label','unlabel']
df.columns = ['Method','Encoder','SuperOnly','unlabel_ratio']
df['best_auc'] = np.array(best)[:,1]
df['last_auc'] = np.array(last)[:,1]
df['best_f1'] = np.array(best)[:,0]
df['last_f1'] = np.array(last)[:,0]

In [6]:
df.sort_values(by='last_auc',ascending=False).reset_index(drop=True)

,Method,Encoder,SuperOnly,unlabel_ratio,best_auc,last_auc,best_f1,last_f1
0,TE5,resnet18,True,0.0,0.7147,0.7037,0.709209,0.697488
1,Pi6,resnet18,False,0.0,0.6802,0.6279,0.674855,0.606610
2,TE6,resnet18,False,0.0,0.6950,0.6267,0.691103,0.613035
3,Pi7,resnet18,False,0.3,0.6724,0.6056,0.660162,0.583257
4,Pi5,resnet18,True,0.0,0.7310,0.6010,0.726405,0.597244
5,TE7,resnet18,False,0.3,0.6542,0.5549,0.648958,0.527469
6,TE8,resnet18,False,0.6,0.6177,0.5246,0.613283,0.512437
7,Pi8,resnet18,False,0.6,0.6210,0.5028,0.600437,0.455598
8,TE9,resnet18,False,0.9,0.5202,0.4834,0.495794,0.459540
9,Pi9,resnet18,False,0.9,0.5739,0.4607,0.560758,0.416529


In [7]:
t = df[df['Method'].apply(lambda x : 'TE' in x )].sort_values(by='best_auc',ascending=False).reset_index(drop=True)
t.drop(columns = ['last_auc',	'best_f1'	,'last_f1'])

,Method,Encoder,SuperOnly,unlabel_ratio,best_auc
0,TE5,resnet18,True,0.0,0.7147
1,TE6,resnet18,False,0.0,0.6950
2,TE7,resnet18,False,0.3,0.6542
3,TE8,resnet18,False,0.6,0.6177
4,TE9,resnet18,False,0.9,0.5202
5,TE0,PiModel,True,0.0,0.2191
6,TE2,PiModel,False,0.3,0.1246
7,TE4,PiModel,False,0.9,0.1108
8,TE1,PiModel,False,0.0,0.1060
9,TE3,PiModel,False,0.6,0.0999
